# Cryptography in Python

### Lets first look at Symmetric key cryptography

In [2]:
from cryptography.fernet import Fernet
class Encryptor():

    def key_create(self):
        key = Fernet.generate_key()
        return key

    def key_write(self, key, key_name):
        with open(key_name, 'wb') as mykey:
            mykey.write(key)

    def key_load(self, key_name):
        with open(key_name, 'rb') as mykey:
            key = mykey.read()
        return key


    def file_encrypt(self, key, original_file, encrypted_file):
        
        f = Fernet(key)

        with open(original_file, 'rb') as file:
            original = file.read()

        encrypted = f.encrypt(original)

        with open (encrypted_file, 'wb') as file:
            file.write(encrypted)

    def file_decrypt(self, key, encrypted_file, decrypted_file):
        
        f = Fernet(key)

        with open(encrypted_file, 'rb') as file:
            encrypted = file.read()

        decrypted = f.decrypt(encrypted)

        with open(decrypted_file, 'wb') as file:
            file.write(decrypted)


In [1]:
# Lets creat a file and store some text on it and encrypt and decrypt it
with open('file.txt','w') as file:
    file.write('Hello, this is secret message')


In [3]:
# now call our methods to get encrypted and decrypted files
encryptor=Encryptor()

mykey=encryptor.key_create()

encryptor.key_write(mykey, 'mykey.key')

loaded_key=encryptor.key_load('mykey.key')

encryptor.file_encrypt(loaded_key, 'file.txt', 'enc_file.txt')

encryptor.file_decrypt(loaded_key, 'enc_file.txt', 'dec_file.txt')


# Calculating Hash of a file in Python

In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

def calculate_file_hash(file_path):
    sha256_hash = hashes.Hash(hashes.SHA256())

    with open(file_path, 'rb') as file:
        while chunk := file.read(8192):
            sha256_hash.update(chunk)

# now finalize our hash
    file_hash = sha256_hash.finalize()

    return file_hash

# Make sure you have this file in the same directory
file_path = 'message.txt'
hash_value = calculate_file_hash(file_path)

print(f"SHA-256 Hash of '{file_path}': {hash_value.hex()}")


#The := is known as the "walrus operator,"
# We can also pass complete file at once to calculate its hash

SHA-256 Hash of 'message.txt': c8ed5e65713f8170b8d0769a8ed8c35740ba0b706d5856e22d709be1e001cf8c
